# Importing data #

In [1]:
import pandas as pd

with open('dialog16/dialog_testset.txt', 'r') as f:
    input_texts = f.readlines()
    input_texts = [i[:-1] for i in input_texts if i != '\n']
    
with open('dialog16/true_dialog_testset.txt', 'r') as f:
    true_texts = f.readlines()
    true_texts = [i[:-1] for i in true_texts if i != '\n']

data = pd.DataFrame(data=zip(input_texts, true_texts), columns=['source', 'true'])
print(len(data))
data.tail(3)

2008


,source,true
2005,и вобщем-то все понятно на фоне слухов застави...,и в общем-то все понятно на фоне слухов застав...
2006,но всему есть придел,но всему есть предел
2007,у нас в доме оччень много застекленных вертика...,у нас в доме очень много застекленных вертикал...


# Tokenizing

In [2]:
data['tokenized_source'] = [text.strip().split() for text in data['source']]
data['tokenized_true'] = [text.strip().split() for text in data['true']]
data.head(2)

,source,true,tokenized_source,tokenized_true
0,есть у вас оформленый и подписаный мною заказ,есть у вас оформленный и подписанный мною заказ,"[есть, у, вас, оформленый, и, подписаный, мною...","[есть, у, вас, оформленный, и, подписанный, мн..."
1,вот в инете откапал такую интеерсную статейку ...,вот в инете откопал такую интересную статейку ...,"[вот, в, инете, откапал, такую, интеерсную, ст...","[вот, в, инете, откопал, такую, интересную, ст..."


# Standart ELMo (5 gb)

In [3]:
from ELMO_inference import ELMOLM

[nltk_data] Downloading package punkt to /home/azat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/azat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/azat/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/azat/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [4]:
standart_elmo = ELMOLM('/home/azat/.deeppavlov/downloads/embeddings/elmo_ru_news',
                       './ru_elmo_voc_scores_by_kenlm.json', True)

Using TensorFlow backend.








Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/azat/.deeppavlov/downloads/embeddings/elmo_ru_news/model.ckpt-1327437


In [5]:
%%time
source_likelihood = standart_elmo.estimate_likelihood_batch(data['tokenized_source'], batch_size=10)
true_likelihood = standart_elmo.estimate_likelihood_batch(data['tokenized_true'], batch_size=10)

201it [1:04:25, 15.80s/it]
201it [1:04:31, 15.89s/it]

CPU times: user 1h 45min 28s, sys: 24min 21s, total: 2h 9min 49s
Wall time: 2h 8min 57s


In [7]:
data['scores_true'] = true_likelihood
data['scores_source'] = source_likelihood

In [8]:
data.to_csv('./answers_basic_model.csv')

In [9]:
ans = data['scores_true'] > data['scores_source']
sum(ans) / len(ans)

0.7559760956175299

# Standart ELMo (5 gb) one track

In [3]:
from ELMO_inference import ELMO_LM_one_track

[nltk_data] Downloading package punkt to /home/azat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/azat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/azat/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/azat/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [4]:
elmo_one_track = ELMO_LM_one_track('/home/azat/.deeppavlov/downloads/embeddings/elmo_ru_news',
                       './ru_elmo_voc_scores_by_kenlm.json', True)

Using TensorFlow backend.








Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/azat/.deeppavlov/downloads/embeddings/elmo_ru_news/model.ckpt-1327437


# Consider hyps with the same size as source

In [5]:
length_source = data['tokenized_source'].apply(lambda x: len(x))
length_true = data['tokenized_true'].apply(lambda x: len(x))
same_size = data[length_source == length_true]
len(same_size)

1680

In [6]:
%%time

hyps = list(zip(same_size['tokenized_source'], same_size['tokenized_true']))
likelihood = elmo_one_track.estimate_likelihood_batch(input_batch=same_size['tokenized_source'],
                                                      hyp_batch=hyps)

168it [52:56, 17.66s/it]

CPU times: user 43min 21s, sys: 9min 55s, total: 53min 17s
Wall time: 52min 56s


In [7]:
same_size['source_likelihood'] =[i[0] for i in likelihood]
same_size['true_likelihood'] =[i[1] for i in likelihood]

/home/azat/anaconda3/envs/spell/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/azat/anaconda3/envs/spell/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
ans = same_size.source_likelihood < same_size.true_likelihood
sum(ans) / len(ans)

0.47023809523809523